In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re


In [2]:
#Get A share stock list
url="http://www.haiguitouzi.com/doc/intro_stock_list.php"
#Requests module use to data from given url
source=requests.get(url)
# BeautifulSoup is used for getting HTML structure from requests response.
soup=BeautifulSoup(source.text,'html')


In [3]:
# Find function is used to find a single element if there are more than once it always returns the first element.
title=soup.find('title')
print("This is with html tags :",title)

#stock list
stock_list=soup.find('tbody')


This is with html tags : <title>A股股票列表 A股所有上市公司股票名称  代码列表——海龟投资</title>


In [4]:
#Extract stock data
rows = stock_list.find_all('tr')
stock_data = []

for row in rows:
    cols = row.find_all('td')
    cols = [col.text.strip() for col in cols]
    
    stock_data.append(cols)

for stock in stock_data:
    print(stock)

['000001平安银行', '000002万科A', '000004国华网安', '000005ST星源', '000006深振业A', '000007全新好']
['000008神州高铁', '000009中国宝安', '000010美丽生态', '000011深物业A', '000012南玻A', '000014沙河股份']
['000016深康佳A', '000017深中华A', '000019深粮控股', '000020深华发A', '000021深科技', '000023*ST深天']
['000025特力A', '000026飞亚达', '000027深圳能源', '000028国药一致', '000029深深房A', '000030富奥股份']
['000031大悦城', '000032深桑达A', '000034神州数码', '000035中国天楹', '000036华联控股', '000037深南电A']
['000039中集集团', '000040ST旭蓝', '000042中洲控股', '000045深纺织A', '000048京基智农', '000049德赛电池']
['000050深天马A', '000055方大集团', '000056皇庭国际', '000058深赛格', '000059华锦股份', '000060中金岭南']
['000061农产品', '000062深圳华强', '000063中兴通讯', '000065北方国际', '000066中国长城', '000068华控赛格']
['000069华侨城A', '000070ST特信', '000078海王生物', '000088盐田港', '000089深圳机场', '000090天健集团']
['000096广聚能源', '000099中信海直', '000100TCL科技', '000151中成股份', '000153丰原药业', '000155川能动力']
['000156华数传媒', '000157中联重科', '000158常山北明', '000159国际实业', '000166申万宏源', '000301东方盛虹']
['000333美的集团', '000338潍柴动力', '000400许继电气', '000401冀东水泥', '000402金融街', '00

In [5]:
complete_list = sum(stock_data, [])
df = pd.DataFrame(complete_list)

# split ticker and company name
def split_ticker_and_name(stock):
    match = re.match(r"(\d+)(\D+)", stock) 
    if match:
        return match.groups()
    return stock, '' 

split_df = df.applymap(split_ticker_and_name)

# new dataframe
ticker_df = split_df.applymap(lambda x: x[0])  # tickers
ticker_df.columns = ['tickers']
company_df = split_df.applymap(lambda x: x[1])  # company names
company_df.columns = ['company']

# complete the final daraframe
final_df = pd.concat([ticker_df, company_df], axis=1)

final_df


/var/folders/1g/kkn6qdxn5jj1lnwwtw1hhwg00000gn/T/ipykernel_30292/513262798.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  split_df = df.applymap(split_ticker_and_name)
/var/folders/1g/kkn6qdxn5jj1lnwwtw1hhwg00000gn/T/ipykernel_30292/513262798.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ticker_df = split_df.applymap(lambda x: x[0])  # tickers
/var/folders/1g/kkn6qdxn5jj1lnwwtw1hhwg00000gn/T/ipykernel_30292/513262798.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  company_df = split_df.applymap(lambda x: x[1])  # company names


,tickers,company
0,000001,平安银行
1,000002,万科A
2,000004,国华网安
3,000005,ST星源
4,000006,深振业A
...,...,...
5394,920002,万达轴承
5395,920008,成电光信
5396,920016,中草香料
5397,920099,瑞华技术


In [6]:
# save to .csv file
final_df.to_csv('stock_data.csv', index=False)